In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Aumento del Tamaño del dataset

Antes de poder generar nuevos datos para el datasets debemos de observar nuestros archivos csv y procesarlos.


## Archivos .csv

### resumen_evolucion.csv

In [20]:
datos_evolucion = pd.read_csv(r"C:\Users\mpord\Documents\3IngSoft\2Cuatri\G4.Dedalus\material_dedalus\resumen_evolucion_process.csv", sep=",", decimal=".", on_bad_lines='skip')

print(datos_evolucion)

    PacienteID       Fecha   Hora  PresionSistolica  PresionDiastolica  \
0            1  01/05/2023  06:00             100.0               65.0   
1            1  01/05/2023  14:00             105.0               68.0   
2            1  01/05/2023  22:00              98.0               60.0   
3            1  02/05/2023  06:00             105.0               70.0   
4            1  02/05/2023  14:00             110.0               72.0   
5            1  02/05/2023  22:00             100.0               68.0   
6            1  03/05/2023  06:00             115.0               75.0   
7            1  03/05/2023  14:00             118.0               78.0   
8            1  03/05/2023  22:00             110.0               70.0   
9            1  04/05/2023  06:00             120.0               80.0   
10           1  04/05/2023  14:00             125.0               82.0   
11           1  04/05/2023  22:00             115.0               78.0   
12           1  05/05/2023  06:00     

In [23]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Cargar los datos
df = datos_evolucion

# Convertir columnas a numéricas
df['LDL'] = pd.to_numeric(df['LDL'], errors='coerce')
df['Trigliceridos'] = pd.to_numeric(df['Trigliceridos'], errors='coerce')
df['Sodio'] = pd.to_numeric(df['Sodio'], errors='coerce')

# Imputar valores faltantes con la mediana (puedes cambiarlo si lo prefieres)
imputer = SimpleImputer(strategy='median')
df[['PresionSistolica', 'PresionDiastolica', 'FrecuenciaCardiaca', 'Temperatura', 'FrecuenciaRespiratoria', 
    'SaturacionOxigeno', 'Glucosa', 'Leucocitos', 'Hemoglobina', 'Plaquetas', 'Colesterol', 'HDL', 'LDL', 
    'Trigliceridos', 'Sodio', 'Potasio', 'Cloro', 'Creatinina', 'Urea', 'AST', 'ALT', 'Bilirrubina', 'pH', 
    'pCO2', 'pO2', 'HCO3', 'Lactato']] = imputer.fit_transform(df[['PresionSistolica', 'PresionDiastolica', 'FrecuenciaCardiaca', 'Temperatura', 
                                                                'FrecuenciaRespiratoria', 'SaturacionOxigeno', 'Glucosa', 'Leucocitos', 
                                                                'Hemoglobina', 'Plaquetas', 'Colesterol', 'HDL', 'LDL', 'Trigliceridos', 
                                                                'Sodio', 'Potasio', 'Cloro', 'Creatinina', 'Urea', 'AST', 'ALT', 
                                                                'Bilirrubina', 'pH', 'pCO2', 'pO2', 'HCO3', 'Lactato']])

# Normalizar las variables numéricas
scaler = MinMaxScaler()
df[['PresionSistolica', 'PresionDiastolica', 'FrecuenciaCardiaca', 'Temperatura', 'FrecuenciaRespiratoria', 
    'SaturacionOxigeno', 'Glucosa', 'Leucocitos', 'Hemoglobina', 'Plaquetas', 'Colesterol', 'HDL', 'LDL', 
    'Trigliceridos', 'Sodio', 'Potasio', 'Cloro', 'Creatinina', 'Urea', 'AST', 'ALT', 'Bilirrubina', 'pH', 
    'pCO2', 'pO2', 'HCO3', 'Lactato']] = scaler.fit_transform(df[['PresionSistolica', 'PresionDiastolica', 'FrecuenciaCardiaca', 'Temperatura', 
                                                                'FrecuenciaRespiratoria', 'SaturacionOxigeno', 'Glucosa', 'Leucocitos', 
                                                                'Hemoglobina', 'Plaquetas', 'Colesterol', 'HDL', 'LDL', 'Trigliceridos', 
                                                                'Sodio', 'Potasio', 'Cloro', 'Creatinina', 'Urea', 'AST', 'ALT', 
                                                                'Bilirrubina', 'pH', 'pCO2', 'pO2', 'HCO3', 'Lactato']])

# Codificar las variables categóricas
df = pd.get_dummies(df, columns=['PacienteID', 'Fecha', 'Hora'])


In [26]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization
import numpy as np

# Definir el tamaño de las entradas y el tamaño del ruido
input_dim = df.shape[1]  # número de características en el conjunto de datos
z_dim = 100  # tamaño del vector de ruido

# Construcción del Generador
def build_generator(z_dim):
    model = Sequential()
    model.add(Dense(128, input_dim=z_dim))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(input_dim, activation='tanh'))  # Activación 'tanh' para normalización
    return model

# Construcción del Discriminador
def build_discriminator(input_dim):
    model = Sequential()
    model.add(Dense(512, input_dim=input_dim))
    model.add(LeakyReLU(0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(0.2))
    model.add(Dense(128))
    model.add(LeakyReLU(0.2))
    model.add(Dense(1, activation='sigmoid'))  # Salida binaria: real o falso
    return model

# Construcción del modelo GAN
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Compilar el Discriminador
discriminator = build_discriminator(input_dim)
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Compilar el Generador
generator = build_generator(z_dim)

# El Discriminador debe estar fijo al entrenar el GAN (no se entrena durante el entrenamiento del generador)
discriminator.trainable = False

# Construir el GAN
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Función para entrenar la GAN
def train_gan(epochs, batch_size, df, generator, discriminator, gan):
    batch_count = df.shape[0] // batch_size

    if batch_count == 0:
        print("Batch size is too large for the dataset.")
        return

    for epoch in range(epochs):
        for _ in range(batch_count):
            # Generar datos falsos
            noise = np.random.normal(0, 1, (batch_size, z_dim))
            generated_data = generator.predict(noise)

            # Obtener una muestra aleatoria de datos reales
            real_data = df.sample(batch_size).values

            # Etiquetas: 1 para datos reales, 0 para generados
            labels_real = np.ones((batch_size, 1))
            labels_fake = np.zeros((batch_size, 1))

            # Entrenar el discriminador
            d_loss_real = discriminator.train_on_batch(real_data, labels_real)
            d_loss_fake = discriminator.train_on_batch(generated_data, labels_fake)

            # Entrenar el generador (la red GAN)
            noise = np.random.normal(0, 1, (batch_size, z_dim))
            g_loss = gan.train_on_batch(noise, labels_real)  # El generador engaña al discriminador

        print(f"Epoch {epoch+1}/{epochs} | D Loss Real: {d_loss_real[0]} | D Loss Fake: {d_loss_fake[0]} | G Loss: {g_loss}")

# Entrenar la GAN
train_gan(epochs=10000, batch_size=16, df=df, generator=generator, discriminator=discriminator, gan=gan)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step


c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\lib\site-packages\keras\src\backend\tensorflow\trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).